In [54]:
import json
import msal
import requests
import webbrowser
import os
# import boto3
import logging
import pandas as pd
import mysql.connector
from mysql.connector import Error, IntegrityError
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from datetime import datetime


In [55]:

# Set up logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Constants for OAuth2.0 authentication and API access
SCOPES = ['User.Read','Mail.Read']
APP_ID = 'a7c0a132-7332-4d7e-a59b-879f4d227bcb'
#
base_tmp_path = "/tmp/"

# Get current UTC time
current_utc_time = datetime.utcnow()
# Format to ISO 8601 string
formatted_time = current_utc_time.strftime('%Y-%m-%dT%H:%M:%SZ')


In [56]:


def generate_access_token(app_id, scopes):
    # Initialize the token cache
    access_token_cache = msal.SerializableTokenCache()
    local_filename_initial = 'api_token_access_arp.json'
    filename = 'api_token_access_arp.json'
    local_file_path = f'{base_tmp_path}{filename}'
    # Load token cache from file if it exists
    if os.path.exists(local_filename_initial):
        access_token_cache.deserialize(open(local_filename_initial, 'r').read())
    # Create MSAL public client application
    client = msal.PublicClientApplication(client_id=app_id, token_cache=access_token_cache)
    accounts = client.get_accounts()

    # Try to acquire token silently
    if accounts:
        token_response = client.acquire_token_silent(scopes, account=accounts[0])
    else:
        # Initiate device flow if no accounts are found
        flow = client.initiate_device_flow(scopes=scopes)
        print('User code :' + flow['user_code'])
        webbrowser.open(flow['verification_uri'])
        token_response = client.acquire_token_by_device_flow(flow)
    # Save the updated token cache to a file
    with open(local_file_path, 'w') as _f:
        _f.write(access_token_cache.serialize())
    return token_response


# Function to get unread emails
def get_emails(access_token):
    endpoint = "https://graph.microsoft.com/v1.0/me/mailFolders/Inbox/messages?$filter=isRead eq false&$top=1000"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(endpoint, headers=headers)
    if response.status_code == 200:
        emails = response.json()
        return emails.get('value', [])
    else:
        raise Exception(f"Error fetching emails: {response.status_code} - {response.text}")
         
# Function to convert HTML to plain text
def html_to_text(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup.get_text()



In [57]:
token_response = generate_access_token(APP_ID, SCOPES)
access_token = token_response['access_token']
emails = get_emails(access_token)
print("LENGTH OF THE RAWS EMAILS: ",len(emails))

LENGTH OF THE RAWS EMAILS:  1000


In [58]:

# Function to post response data
def post_queries_to_api(access_token,json_data):
    # Define the API endpoint and headers for the POST request
    api_url = 'https://staging.jsjdmedia.com/api/get-filter'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    try:
        # Send the POST request with the JSON data and headers
        response = requests.post(api_url, headers=headers, json=json_data)
        response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)
        print('Data successfully sent to API.')
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response JSON
            response_data = response.json()
            print("Response JSON:", json.dumps(response_data, indent=4))
        return response_data
    except requests.exceptions.HTTPError as err:
        # Handle HTTP errors
        print(f'HTTP error occurred: {err}')
        return {
            'statusCode': 500,
            'body': json.dumps(f'HTTP error occurred: {err}')
        }
    except Exception as e:
        # Handle other exceptions
        print(f'Other error occurred: {e}')
        return {
            'statusCode': 500,
            'body': json.dumps(f'Other error occurred: {e}')
        }

In [59]:
# Use a set comprehension to gather unique email addresses
email_address_set = {email.get('from', {}).get('emailAddress', {}).get('address')
    for email in emails if email.get('from', {}).get('emailAddress', {}).get('address')}
    
# Create the dictionary in one step using list comprehension
email_dict = {"emailList": [{str(1): email} for i, email in enumerate(email_address_set)]}
print(f"email_dict----------{email_dict}")


filter_data = post_queries_to_api(access_token,email_dict)
print("Filter data:",filter_data)

email_dict----------{'emailList': [{'1': 'case@arpnewsletters.com'}, {'1': 'txoga@associationrevenuepartners.com'}, {'1': 'sburton@bwgc.net'}, {'1': 'alobozar@motic-america.com'}, {'1': 'arosa@reedexpo.com'}, {'1': 'anniexu@itweae.com'}, {'1': 'matthew.banker@gmail.com'}, {'1': 'a.zubov@farmak.ua'}, {'1': 'scmbede@gmail.com'}, {'1': 'candi@crsdc.com'}, {'1': 'genebeck@sweettoothfamilydental.com'}, {'1': 'laurence.lebourhis@guerbet.com'}, {'1': 'm.almukhamedova@globalinklogistics.com'}, {'1': 'contact@t.brevo.com'}, {'1': 'noreply@account.tiktok.com'}, {'1': 'support@fetchfind.zohodesk.com'}, {'1': 'nhung@nime.jp'}, {'1': 'frances@levyaa.com'}, {'1': 'cpta@associationrevenuepartners.com'}, {'1': 'cottendustleisure@cottendust.com'}, {'1': 'christopher.regan@teneoholdings.com'}, {'1': 'hfm@industry411.com'}, {'1': 'noreply-Industry411-Internal-Message@x4.gmobb.jp'}, {'1': 'hr@bcit.ca'}, {'1': 'noreply@jotform.com'}, {'1': 'acoe@associationrevenuepartners.com'}, {'1': 'ScannerDocxAssociati

In [62]:
filter_data

[{'1': 'case@arpnewsletters.com'},
 {'1': 'txoga@associationrevenuepartners.com'},
 {'1': 'sburton@bwgc.net'},
 {'1': 'alobozar@motic-america.com'},
 {'1': 'arosa@reedexpo.com'},
 {'1': 'anniexu@itweae.com'},
 {'1': 'matthew.banker@gmail.com'},
 {'1': 'a.zubov@farmak.ua'},
 {'1': 'scmbede@gmail.com'},
 {'1': 'candi@crsdc.com'},
 {'1': 'genebeck@sweettoothfamilydental.com'},
 {'1': 'laurence.lebourhis@guerbet.com'},
 {'1': 'm.almukhamedova@globalinklogistics.com'},
 {'1': 'contact@t.brevo.com'},
 {'1': 'noreply@account.tiktok.com'},
 {'1': 'support@fetchfind.zohodesk.com'},
 {'1': 'nhung@nime.jp'},
 {'1': 'frances@levyaa.com'},
 {'1': 'cpta@associationrevenuepartners.com'},
 {'1': 'cottendustleisure@cottendust.com'},
 {'1': 'christopher.regan@teneoholdings.com'},
 {'1': 'hfm@industry411.com'},
 {'1': 'noreply-Industry411-Internal-Message@x4.gmobb.jp'},
 {'1': 'hr@bcit.ca'},
 {'1': 'noreply@jotform.com'},
 {'1': 'acoe@associationrevenuepartners.com'},
 {'1': 'ScannerDocxAssociationrevenu

In [42]:
def process_emails(email, filter_id, filter_name, filter_is_active):
    data = []
    subject = email['subject']
    sender_address = email['sender']['emailAddress']['address']
    full_body_html = email['body']['content']
    email_body_plain = html_to_text(full_body_html)
    # Check if the email matches the filter criteria
    if filter_name in subject or filter_name in email_body_plain:
        changeKey = email['changeKey']
        received_info = email['receivedDateTime']
        send_info = email['sentDateTime']
        try:
            sql = f"""INSERT INTO filters_mail_data (filterTableId, mailTo, sentTime, receiveTime, changeKey, subject, mailBody, mail_status, error_code, created_at, updated_at)
            VALUES ({filter_id}, '{sender_address}', '{send_info}', '{received_info}', '{changeKey}', '{subject}', '{email_body_plain}', {filter_is_active}, 1, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)"""
            print(f"SQL Query: {sql}")

            # cursor = connection.cursor()
            print("Cursor initialized")

            # cursor.execute(sql)
            print("SQL executed")

            # connection.commit()
            print("Transaction committed")

            print("Record inserted successfully")

        except IntegrityError as e:
            print(f"IntegrityError: {e}")
            # Handle duplicate key error
            if e.errno == 1062:  # 1062 is the error code for duplicate entry
                print(f"Duplicate entry for changeKey: {changeKey}. Record not inserted.")
            else:
                # Handle other IntegrityError exceptions
                print(f"IntegrityError: {e}")

    # connection.close()
    print("Connection closed")
    return


In [61]:
filtered_data = []
    # Process input data
for item in filter_data:
    # Check if filterData exists in the current item
    if "filterData" in item:
        print((True))
        filtered_data.append(item)   
print(filtered_data)
# for i in filtered_data:
#     print(i['1'])
for email in emails:
    try:
        email_address = email['from']['emailAddress']['address']
        # print(email_address)
        
        for filter in filtered_data:
            # print(filter)
            if filter['1'] == email_address:
                # print(email_address)
                for filter_item in filter['filterData']:
                    filter_info = filter_item['filterInfo']
                    filter_id = filter_info['id']
                    filter_name = filter_info['filterName']
                    filter_is_active = filter_info['is_active']
                    # process_emails(email, filter_id,filter_name,filter_is_active)
    except Exception as e:
        continue

True
True
[{'1': 'postmaster@NationPizza.com', 'filterData': [{'id': 4, 'emailId': 3, 'filterId': 2, 'tableId': 5, 'is_active': 1, 'deleted_at': None, 'created_at': '2024-07-02T11:32:42.000000Z', 'updated_at': '2024-07-02T11:32:42.000000Z', 'tableInfo': {'id': 5, 'tableName': 'Undeliverable'}, 'filterInfo': {'id': 2, 'filterName': 'Undeliverable:', 'filterOnSubject': 1, 'filterOnBody': 1, 'is_active': 1}}]}, {'1': 'MAILER-DAEMON@GAGGLE.NET', 'filterData': [{'id': 3, 'emailId': 2, 'filterId': 3, 'tableId': 4, 'is_active': 1, 'deleted_at': None, 'created_at': '2024-07-02T11:32:27.000000Z', 'updated_at': '2024-07-02T11:32:27.000000Z', 'tableInfo': {'id': 4, 'tableName': 'Message Failure:'}, 'filterInfo': {'id': 3, 'filterName': 'Message Failure:', 'filterOnSubject': 1, 'filterOnBody': 1, 'is_active': 1}}, {'id': 5, 'emailId': 2, 'filterId': 2, 'tableId': 4, 'is_active': 1, 'deleted_at': None, 'created_at': '2024-07-02T11:32:53.000000Z', 'updated_at': '2024-07-02T11:32:53.000000Z', 'tableI

In [35]:
import pandas as pd

In [45]:
df = pd.read_csv('/home/dotsquares/Desktop/krishna/js_jd_media/msgraph-training-python/graphtutorial/Task1/filter (1).csv')

In [46]:
df

,Unnamed: 0,id,filterTableId,mailTo,sentTime,changeKey,receiveTime,subject,mailBody,mail_status,error_code,deleted_at,created_at,updated_at
0,0,5,3,MAILER-DAEMON@GAGGLE.NET,2024-07-02T12:35:28Z,CQAAABYAAACY28Rccs6eQ6vSFsjSkG/hAAD+UQt7,2024-07-02T12:35:35Z,Message Failure: AI for Special Ed: Free Serie...,\nYour message could not be delivered. The rea...,1,1,NaN,2024-07-03 08:35:41,2024-07-03 08:35:41
1,1,6,3,MAILER-DAEMON@GAGGLE.NET,2024-07-02T12:35:10Z,CQAAABYAAACY28Rccs6eQ6vSFsjSkG/hAAD+UQty,2024-07-02T12:35:16Z,Message Failure: AI for Special Ed: Free Serie...,\nYour message could not be delivered. The rea...,1,1,NaN,2024-07-03 08:35:41,2024-07-03 08:35:41
2,2,7,3,MAILER-DAEMON@GAGGLE.NET,2024-06-25T12:32:48Z,CQAAABYAAACY28Rccs6eQ6vSFsjSkG/hAAD5zTh6,2024-06-25T12:32:59Z,Message Failure: New CASE Policy: Enhancing Su...,\nYour message could not be delivered. The rea...,1,1,NaN,2024-07-03 08:35:41,2024-07-03 08:35:41
3,3,8,3,MAILER-DAEMON@GAGGLE.NET,2024-06-25T12:32:22Z,CQAAABYAAACY28Rccs6eQ6vSFsjSkG/hAAD5zThx,2024-06-25T12:32:29Z,Message Failure: New CASE Policy: Enhancing Su...,\nYour message could not be delivered. The rea...,1,1,NaN,2024-07-03 08:35:41,2024-07-03 08:35:41
4,4,9,3,MAILER-DAEMON@GAGGLE.NET,2024-06-20T18:03:33Z,CQAAABYAAACY28Rccs6eQ6vSFsjSkG/hAAD1PTCu,2024-06-20T18:03:40Z,Message Failure: New Research Report: Inside IEP,\nYour message could not be delivered. The rea...,1,1,NaN,2024-07-03 08:35:41,2024-07-03 08:35:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,80,85,2,postmaster@NationPizza.com,2024-04-02T15:02:53Z,FwAAABYAAACY28Rccs6eQ6vSFsjSkG/hAADCVd5A,2024-04-02T15:02:58Z,Undeliverable: Monetize Your Production,\nDelivery has failed to these recipients or g...,1,1,NaN,2024-07-03 08:35:41,2024-07-03 08:35:41
81,81,86,2,postmaster@NationPizza.com,2024-04-02T15:00:59Z,FwAAABYAAACY28Rccs6eQ6vSFsjSkG/hAADCVdxb,2024-04-02T15:01:07Z,Undeliverable: Monetize Your Production,\nDelivery has failed to these recipients or g...,1,1,NaN,2024-07-03 08:35:41,2024-07-03 08:35:41
82,82,87,3,MAILER-DAEMON@GAGGLE.NET,2024-04-02T12:33:12Z,CQAAABYAAACY28Rccs6eQ6vSFsjSkG/hAADCVdel,2024-04-02T12:33:18Z,Message Failure: Teacher Effectiveness Study f...,\nYour message could not be delivered. The rea...,1,1,NaN,2024-07-03 08:35:41,2024-07-03 08:35:41
83,83,88,3,MAILER-DAEMON@GAGGLE.NET,2024-03-26T12:34:50Z,CQAAABYAAACY28Rccs6eQ6vSFsjSkG/hAAC91xCZ,2024-03-26T12:34:57Z,Message Failure: Utah's Alt Path to Special Ed...,\nYour message could not be delivered. The rea...,1,1,NaN,2024-07-03 08:35:41,2024-07-03 08:35:41


In [49]:
len(set(df['changeKey']))

85

In [63]:
email_dict = {'emailList': [{'1': 'case@arpnewsletters.com'}, {'1': 'txoga@associationrevenuepartners.com'}, {'1': 'sburton@bwgc.net'}, {'1': 'alobozar@motic-america.com'}, {'1': 'arosa@reedexpo.com'}, {'1': 'anniexu@itweae.com'}, {'1': 'matthew.banker@gmail.com'}, {'1': 'a.zubov@farmak.ua'}, {'1': 'scmbede@gmail.com'}, {'1': 'candi@crsdc.com'}, {'1': 'genebeck@sweettoothfamilydental.com'}, {'1': 'laurence.lebourhis@guerbet.com'}, {'1': 'm.almukhamedova@globalinklogistics.com'}, {'1': 'contact@t.brevo.com'}, {'1': 'noreply@account.tiktok.com'}, {'1': 'support@fetchfind.zohodesk.com'}, {'1': 'nhung@nime.jp'}, {'1': 'frances@levyaa.com'}, {'1': 'cpta@associationrevenuepartners.com'}, {'1': 'cottendustleisure@cottendust.com'}, {'1': 'christopher.regan@teneoholdings.com'}, {'1': 'hfm@industry411.com'}, {'1': 'noreply-Industry411-Internal-Message@x4.gmobb.jp'}, {'1': 'hr@bcit.ca'}, {'1': 'noreply@jotform.com'}, {'1': 'acoe@associationrevenuepartners.com'}, {'1': 'ScannerDocxAssociationrevenuepartners-EMailServerMailerDenomDeliveryMicrosoftTeamsMeeting-AuthrnicrosftonprotectionutomationoMultFi.Secureactiomfaautomatednotification.Noreply-eml-rnicrosftonprotectionutomationonsecure-mcsfsoft.com.msx-noreply@klotzelectricllc.com'}, {'1': 'popkoornm@yahoo.com'}, {'1': 'no-reply@QuessGTS.com'}, {'1': 'info.notification@cochlear.com'}, {'1': 'aaenp@associationrevenuepartners.com'}, {'1': 'Xinhua_Pan@mccormick.com.cn'}, {'1': 'asps@associationrevenuepartners.com'}, {'1': 'MAILER-DAEMON@vnotify2.communitect.com'}, {'1': 'rmkountz@bgcaa.com'}, {'1': 'Chiara.Zaccarini@fattoriecremona.it'}, {'1': 'itsupport@saltech.com'}, {'1': 'post.master@noreply.chiesi.com'}, {'1': 'DoesNotExist@hodgsonruss.com'}, {'1': 'drdparkin@bigpond.com'}, {'1': 'pecan17@hotmail.com'}, {'1': 'ludovic.lair@entosystem.com'}, {'1': 'ekyn@basikwall.com'}, {'1': 'jqvarnstrom@opentext.com'}, {'1': 'noreply-Industry411-Internal-Message@ao2.gmobb.jp'}, {'1': 'postmaster@MYSRHS.COM'}, {'1': 'alessandro.gamannossi@esaote.com'}, {'1': 'appa@associationrevenuepartners.com'}, {'1': 'djquinn@mgb.org'}, {'1': 'Vivian.Xiao@umusic.com'}, {'1': 'michael@uicgroup.com.tw'}, {'1': 'ceo@ifmrs.org'}, {'1': 'ibpsa@associationrevenuepartners.com'}, {'1': 'info@chaniljungchiro.com'}, {'1': '373602476@qq.com'}, {'1': 'rabedilb@gmail.com'}, {'1': 'txoga@arpnewsletters.com'}, {'1': 'iddba@arpnewsletters.com'}, {'1': 'aaspa@associationrevenuepartners.com'}, {'1': 'elisabeth.champagne@umanomedical.com'}, {'1': 'bbanovich@mklm.org'}, {'1': 'info@uptownrestoration.com'}, {'1': 'aleksandra.novoselova@bienair.com'}, {'1': 'no-reply@enguard.com'}, {'1': 'pinterest-recommendations@ideas.pinterest.com'}, {'1': 'drjones@thompsonlaneeyecare.com'}, {'1': 'erik@baywaveagency.com'}, {'1': 'aic@associationrevenuepartners.com'}, {'1': 'randomuser@nifty.com'}, {'1': 'help@doctormultimedia.com'}, {'1': 'amy.wilson@gowbs.com'}, {'1': 'eSign-Document-Shared-Organization-Internal-Message@lifestartupessentials.com'}, {'1': 'kaellyn@gmail.com'}, {'1': 'andrea@hi-velocity.com'}, {'1': 'tbilliot@nees.us.com'}, {'1': 'hhamilton@azlechristianschool.org'}, {'1': 'dbenoit@novali.ca'}, {'1': 'jybacle@movexinnovation.com'}, {'1': 'mail@activedgetechnologies.com'}, {'1': 'fadi@catherinekwong.com'}, {'1': 'michelle@idscga.com'}, {'1': 'noreply@wiley-epic.com'}, {'1': 'info@humanware.com'}, {'1': 'cedric.pascual@demeco-group.com'}, {'1': 'tps@associationrevenuepartners.com'}, {'1': 'postmaster@outlook.com'}, {'1': 'jboivin@legaultgroup.com'}, {'1': 'noreply@postmaster.scoop.it'}, {'1': 'recommendations@inspire.pinterest.com'}, {'1': 'Megan.Blood@hasbro.com'}, {'1': 'sophie.lefebvre@con-v-air.com'}, {'1': 'davidkurtz@dciproducts.com'}, {'1': 'dp@logcomex.com'}, {'1': 'walshl@oldwestbury.edu'}, {'1': 'sally.weiner@plfdpl.info'}, {'1': 'pnoeldner@nasonvilledairy.com'}, {'1': 'Guenhoe.Lee@britishcouncil.org'}, {'1': 'Cindy.Parres@bclplaw.com'}, {'1': 'benoit.boucher@lassonde.com'}, {'1': 'chris@beltwaysupply.com'}, {'1': 'cindy.fong@boehringer-ingelheim.com'}, {'1': 'michael.li@materialise.com.cn'}, {'1': 'Noreply-UM@unimedizin-mainz.de'}, {'1': 'hisato511@nifty.com'}, {'1': 'stephane.auclair@ametek.com'}, {'1': 'jbrewer.info@wsws.org'}, {'1': 'bella.oh@goyangcvb.com'}, {'1': 'ichemin@biosynex.com'}, {'1': 'pallmar@auction4.gmobb.jp'}, {'1': 'saj@greatexposure.com'}, {'1': 'mhartman1@sjshospitality.com'}, {'1': 'dsummitt@glnd.k12.va.us'}, {'1': 'admissions@nysid.edu'}, {'1': 'cihli@sagrada.design'}, {'1': 'abuse@mxtoolbox.com'}, {'1': 'ilapps@associationrevenuepartners.com'}, {'1': 'ScannerDocxIndustry411-EMailServerMailerDenomDeliveryMicrosoftTeamsMeeting-AuthrnicrosftonprotectionutomationoMultFi.Secureactiomfaautomatednotification.Noreply-eml-rnicrosftonprotectionutomationonsecure-mcsfsoft.com.msx-noreply@onedelisle.com'}, {'1': 'Celine.SEMECAS@Teledyne.com'}, {'1': 'hr@meltbarandgrilled.com'}, {'1': 'jmink@ontario.k12.or.us'}, {'1': 'plende@expedition-water.com'}, {'1': 'elise.bioche@transitions.com'}, {'1': 'its-no-reply@fsu.edu'}, {'1': 'info@haraken.co.jp'}, {'1': 'm.levesque@inovaweld.com'}, {'1': 'unsubscribe@vaderetro-safeunsubscribe.com'}, {'1': 'bhoberman@keystonecyberprotection.com'}, {'1': 'enotis@keystonecyberprotection.com'}, {'1': 'Joelle.M-Lessard@umanomedical.com'}, {'1': 'ava@associationrevenuepartners.com'}, {'1': 'support@spokespace.com'}, {'1': 'isac@associationrevenuepartners.com'}, {'1': 'chloe.bergamaschi@sopexa.com'}, {'1': 'mmcgill@libertyelectronics.com'}, {'1': 'Florian.Piquet@dentsplysirona.com'}, {'1': 'j.khan@meesekhanllp.com'}, {'1': 'trevor@trevorfulmerdesign.com'}, {'1': 'wquow@cwsasvg.com'}, {'1': 'MHachana@fontainesante.com'}, {'1': 'pcourtemanche@pallottihs.org'}, {'1': 'hangil@starmed4u.com'}, {'1': 'lweldy@tabernaclechristian.org'}, {'1': 'a.frenzer@hin.ch'}, {'1': 'rbenswanger@kentcommercialinc.com'}, {'1': 'postmaster@NationPizza.com'}, {'1': 'sethweaver@weaversofwellsville.com'}, {'1': 'lrey@soufflet.com'}, {'1': 'george@conservaco.com'}, {'1': 'publishing@associationrevenuepartners.com'}, {'1': 'adriana.kersting@sap.com'}, {'1': 'liz@lizmurraydesign.com'}, {'1': 'aocoo-hns@associationrevenuepartners.com'}, {'1': 'catie.henoch@klhomestore.com'}, {'1': 'rsamuels@meadowlandsdiaglabs.com'}, {'1': 'clintonmoats@azeementerprises.net'}, {'1': 'MAILER-DAEMON@mysrhs.com'}, {'1': 'fslaoui@pmb-alcen.com'}, {'1': 'jbaker@wakunaga.com'}, {'1': 'tascs@associationrevenuepartners.com'}, {'1': 'jbetts@evergreencountryday.org'}, {'1': 'mgabe@islabella.com'}, {'1': 'cstgermain@flexpipeinc.com'}, {'1': 'luke@squeaky.co'}, {'1': 'ava@arpnewsletters.com'}, {'1': 'Martin.Bartyzal@ellacs.eu'}, {'1': 'info@makeperfectconstruction.co.uk'}, {'1': 'pyvesb@berger.ca'}, {'1': 'mailmarshal@ci.staunton.va.us'}, {'1': 'recommendations@explore.pinterest.com'}, {'1': 'no-reply@mail.instagram.com'}, {'1': 'genevieve@rc-architects.com'}, {'1': 'invitations@alignable.com'}, {'1': 'audrey.deriau@kavokerr.com'}, {'1': 'Noreply_DocX@portllghting.com'}, {'1': 'Denis.Perron@adfastcorp.com'}, {'1': 'aehrbar@mshosp.org'}, {'1': 'Lisa.Berardo@twiningsusa.com'}, {'1': 'talia.pate@feathrapp.com'}, {'1': 'daved@nkwsd.com'}, {'1': 'noreply@cytekbio.com'}, {'1': 'b.mckay@jonesnaturalchews.com'}, {'1': 'Alessia.Bernardo@rothoblaas.com'}, {'1': 'valerie.domeneghetty@guerbet.com'}, {'1': 'iafc@associationrevenuepartners.com'}, {'1': 'jen@wflegal.law'}, {'1': '416959162@qq.com'}, {'1': 'no-reply@dropbox.com'}, {'1': 'nickkelleher@go4thegoal.org'}, {'1': 'recommendations@discover.pinterest.com'}, {'1': 'mimi@mdsnjny.com'}, {'1': 'lisa.parrott@telnet-inc.com'}, {'1': 'h.owen@stanford.edu'}, {'1': 'clientserv@yourmembership.com'}, {'1': 'lanierp@highlands.k12.fl.us'}, {'1': 'tmbrotzman@gmail.com'}, {'1': 'theresa@weblinxinc.com'}, {'1': 'hello@tachmesmd.com'}, {'1': 'nicola.bradburnfuller@eplannerpro.com'}, {'1': 'appa@arpnewsletters.com'}, {'1': 'drritarae@yahoo.com'}, {'1': 'evilleneuve@boulart.com'}, {'1': 'pnadeau@american-biltrite.com'}, {'1': 'gseamer@booksinc.net'}, {'1': 'studio@originalrecord.com'}, {'1': 'MAILER-DAEMON@coroep.com'}, {'1': 'JenyaD@hadassah.org.il'}, {'1': 'no-reply@service.tiktok.com'}, {'1': 'cheryl.pearson@trakamerica.com'}, {'1': 'flempert@cordonbleu.edu'}, {'1': 'askdrmorea@nc.rr.com'}, {'1': 'sbarnett@bewellctr.org'}, {'1': 'iddba@associationrevenuepartners.com'}, {'1': 'steph@zonedisplaycases.com'}, {'1': 'hidetaka0427@qc.commufa.jp'}, {'1': 'FAPFK@tbz.t-com.ne.jp'}, {'1': 'Internet@csjboston.org'}, {'1': 'kimberlyv@hsp-inc.com'}, {'1': 'register@account.tiktok.com'}, {'1': 'omar_tag@hotmail.com'}, {'1': 'rickr@ebaa.com'}, {'1': 'carllevi@epbfi.com'}, {'1': 'ddw@associationrevenuepartners.com'}, {'1': 'mcclellh@highlands.k12.fl.us'}, {'1': 'case@associationrevenuepartners.com'}, {'1': 'fslaoui@theryq.com'}, {'1': 'dan@intwcg.com'}, {'1': 'Filip.Samek@ellacs.eu'}, {'1': 'kai.chen@quidelortho.com'}, {'1': 'MAILER-DAEMON@GAGGLE.NET'}, {'1': 'support@fastiep.com'}, {'1': 'Guilherme_Pimentel@edwards.com'}, {'1': 'momen@nlc.com.sa'}, {'1': 'Lart@grupomyt.com'}, {'1': 'mesa.communications@one-mesa.com'}, {'1': 'sdesilets@technimount.com'}, {'1': 'dsykes@associationrevenuepartners.com'}, {'1': 'olga.moskaeva@msd.com'}, {'1': 'Dominique.Dube@groupenovatech.com'}, {'1': 'BLUKAS@methodisthospitals.org'}, {'1': 'jkennerley@lozanosmith.com'}, {'1': 'kmathi@sasd.us'}, {'1': 'bmazzone@aclsys.org'}, {'1': 'abuse@jsjdmedia.com'}, {'1': 'bbobe@vincennes.in.gov'}, {'1': 'Maija.Masalin@lunawood.com'}, {'1': 'Anat.Z@danya-cebus.co.il'}, {'1': 'support@visiblebody.zendesk.com'}, {'1': 'Jean-Bernard.Gelinas@bridor.com'}, {'1': 'jordan@patchmypc.com'}, {'1': 'MAILER-DAEMON@vnotify1.communitect.com'}]}


In [64]:
email_dict

{'emailList': [{'1': 'case@arpnewsletters.com'},
  {'1': 'txoga@associationrevenuepartners.com'},
  {'1': 'sburton@bwgc.net'},
  {'1': 'alobozar@motic-america.com'},
  {'1': 'arosa@reedexpo.com'},
  {'1': 'anniexu@itweae.com'},
  {'1': 'matthew.banker@gmail.com'},
  {'1': 'a.zubov@farmak.ua'},
  {'1': 'scmbede@gmail.com'},
  {'1': 'candi@crsdc.com'},
  {'1': 'genebeck@sweettoothfamilydental.com'},
  {'1': 'laurence.lebourhis@guerbet.com'},
  {'1': 'm.almukhamedova@globalinklogistics.com'},
  {'1': 'contact@t.brevo.com'},
  {'1': 'noreply@account.tiktok.com'},
  {'1': 'support@fetchfind.zohodesk.com'},
  {'1': 'nhung@nime.jp'},
  {'1': 'frances@levyaa.com'},
  {'1': 'cpta@associationrevenuepartners.com'},
  {'1': 'cottendustleisure@cottendust.com'},
  {'1': 'christopher.regan@teneoholdings.com'},
  {'1': 'hfm@industry411.com'},
  {'1': 'noreply-Industry411-Internal-Message@x4.gmobb.jp'},
  {'1': 'hr@bcit.ca'},
  {'1': 'noreply@jotform.com'},
  {'1': 'acoe@associationrevenuepartners.com'